In [1]:
# Installiere notwendige Pakete
!pip install stable-baselines3[extra] gymnasium shimmy>=2.0.0 quantstats pandas numpy matplotlib yfinance

In [2]:
import os
from google.colab import drive

# 1. Drive mounten
drive.mount('/content/drive')

# --- KONFIGURATION BITTE ANPASSEN ---
# Pfad zu dem Ordner in deinem Drive, wo du das Modell und die CSV hingelegt hast
BASE_DIR = "/content/drive/MyDrive/02_DataScience_Quant/code/Seminar Deep Learning/PPO_Portfolio_Optimization/"

MODEL_PATH = os.path.join(BASE_DIR, "models/best_model.zip")
DATA_PATH = os.path.join(BASE_DIR, "data/processed/features_cleaned.csv") # Updated based on user input

# Evaluierungs-Zeitraum
EVAL_START = '2020-01-01'
EVAL_END = '2023-12-31'
WINDOW_SIZE = 30
INITIAL_BALANCE = 10000.0
TRANSACTION_COST = 0.0005 # 0.05%

# Check ob Dateien gefunden werden
if os.path.exists(MODEL_PATH) and os.path.exists(DATA_PATH):
    print("✅ Dateien gefunden!")
else:
    print(f"❌ FEHLER: Dateien nicht gefunden in {BASE_DIR}")
    print(f"Suche Modell hier: {MODEL_PATH}")
    print(f"Suche Daten hier: {DATA_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dateien gefunden!


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# --- 1. Feature Extractor ---
class CustomCombinedExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, extractor_type="LSTM", hidden_size=128, rnn_dropout=0.0):
        market_space = observation_space["market_data"]
        weights_space = observation_space["portfolio_weights"]
        features_dim = hidden_size + weights_space.shape[0]
        super().__init__(observation_space, features_dim)

        self.lstm = nn.LSTM(
            input_size=market_space.shape[1] * market_space.shape[2],
            hidden_size=hidden_size,
            num_layers=2,
            dropout=rnn_dropout,
            batch_first=True
        )

    def forward(self, observations):
        market = observations["market_data"]
        weights = observations["portfolio_weights"]
        batch_size = market.shape[0]
        market_flat = market.reshape(batch_size, market.shape[1], -1)
        _, (hidden, _) = self.lstm(market_flat)
        return torch.cat([hidden[-1], weights], dim=1)

# --- 2. Environment (MIT HISTORY TRACKING FÜR DASHBOARD) ---
class PortfolioEnv(gym.Env):
    def __init__(self, df, start_date, end_date, window_size, initial_balance, cost_pct):
        super().__init__()
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.cost_pct = cost_pct

        # Daten Laden & Reshapen
        self._load_data(df)
        self._set_time_slices(start_date, end_date)

        # Spaces
        self.action_space = spaces.Box(low=-20, high=20, shape=(self.num_assets + 1,), dtype=np.float32)
        self.observation_space = spaces.Dict({
            "market_data": spaces.Box(low=-np.inf, high=np.inf, shape=(window_size, self.num_assets, self.num_features), dtype=np.float32),
            "portfolio_weights": spaces.Box(low=0, high=1, shape=(self.num_assets + 1,), dtype=np.float32),
        })

    def _load_data(self, df):
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])

        try:
            if 'Ticker' in df.columns:
                # Long Format -> Wide Format
                df_wide = df.set_index(['Date', 'Ticker']).unstack(level='Ticker')
                self.feature_names = df_wide.columns.levels[0].tolist()
                self.asset_names = df_wide.columns.levels[1].tolist()

                df_wide = df_wide.ffill().bfill()
                self.full_data_index = df_wide.index

                vals = df_wide.stack(level='Ticker', future_stack=True).values
                self.num_assets = len(self.asset_names)
                self.num_features = len(self.feature_names)
                self._market_data = vals.reshape(len(df_wide), self.num_assets, self.num_features).astype(np.float32)

                try:
                    self.ret_idx = next(i for i, c in enumerate(self.feature_names) if 'Return' in c or 'pct' in c)
                except StopIteration:
                    self.ret_idx = 0
            else:
                raise ValueError("CSV muss 'Ticker' Spalte haben.")
        except Exception as e:
            print(f"Fehler beim Datenladen: {e}")
            raise e

    def _set_time_slices(self, start, end):
        start_ts = pd.to_datetime(start)
        end_ts = pd.to_datetime(end)
        idxs = self.full_data_index.searchsorted([start_ts, end_ts])
        self.start_step = max(idxs[0], self.window_size)
        self.end_step = min(idxs[1], len(self.full_data_index) - 1)

    def reset(self, seed=None):
        self.current_step = self.start_step
        self.portfolio_value = self.initial_balance
        self.weights = np.zeros(self.num_assets + 1, dtype=np.float32)
        self.weights[0] = 1.0 # 100% Cash

        # --- HISTORY TRACKING (Erweitert) ---
        self.portfolio_history = [self.initial_balance]
        self.date_history = [self.full_data_index[self.current_step]]
        self.weights_history = [self.weights.copy()]

        # Neue Metriken für Dashboard
        self.cost_history = [0.0]
        self.turnover_history = [0.0]
        self.daily_return_history = [0.0]
        self.mkt_return_history = [0.0]
        # ------------------------------------

        return self._get_obs(), {}

    def _get_obs(self):
        return {
            "market_data": self._market_data[self.current_step-self.window_size:self.current_step],
            "portfolio_weights": self.weights
        }

    def step(self, action):
        # Softmax Action -> Weights
        exp = np.exp(np.clip(action, -20, 20))
        new_weights = exp / np.sum(exp)

        # Transaktionskosten
        turnover = np.sum(np.abs(new_weights[1:] - self.weights[1:]))
        cost = turnover * self.portfolio_value * self.cost_pct

        # Rendite berechnen
        asset_rets = self._market_data[self.current_step, :, self.ret_idx]

        # Portfolio Return & Markt Benchmark Return
        port_ret = np.sum(new_weights[1:] * asset_rets)
        mkt_ret = np.mean(asset_rets) # Equal Weight Benchmark des Universums

        # Neuer Wert
        self.portfolio_value = (self.portfolio_value - cost) * (1 + port_ret)

        # Update State
        self.current_step += 1
        self.weights = new_weights

        # --- HISTORY UPDATES ---
        self.portfolio_history.append(self.portfolio_value)
        self.date_history.append(self.full_data_index[self.current_step])
        self.weights_history.append(self.weights.copy())

        self.cost_history.append(cost)
        self.turnover_history.append(turnover)
        self.daily_return_history.append(port_ret)
        self.mkt_return_history.append(mkt_ret)
        # -----------------------

        done = self.current_step >= self.end_step
        return self._get_obs(), 0, done, False, {}

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import matplotlib.pyplot as plt

# Setzt das Matplotlib-Design auf einen dunklen Modus
plt.style.use('dark_background')

# Optional: Anpassung der Hintergrundfarbe der Plots selbst (falls nötig)
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['axes.facecolor'] = 'black'

# ... danach Ihr normaler Code ...
# qs.reports.html(returns, benchmark=benchmark, output='QuantStats_Report.html', title='PPO Agent Evaluation')

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [6]:
import quantstats as qs
import matplotlib.pyplot as plt
from IPython.display import FileLink

def run_evaluation():
    print("1. Lade Daten...")
    df = pd.read_csv(DATA_PATH)

    print("2. Erstelle Environment...")
    env = PortfolioEnv(df, EVAL_START, EVAL_END, WINDOW_SIZE, INITIAL_BALANCE, TRANSACTION_COST)

    print("3. Lade Modell...")

    # Architektur Rekonstruktion (wie im Training)
    policy_kwargs = dict(
        features_extractor_class=CustomCombinedExtractor,
        features_extractor_kwargs=dict(
            hidden_size=128,
            rnn_dropout=0.0
        ),
        net_arch=dict(pi=[64], vf=[64])
    )

    model = PPO.load(
        MODEL_PATH,
        custom_objects={
            "learning_rate": 0.0,
            "lr_schedule": lambda _: 0.0,
            "clip_range": lambda _: 0.1,
            "policy_kwargs": policy_kwargs
        }
    )

    print("4. Starte Backtest (das kann kurz dauern)...")
    obs, _ = env.reset()
    done = False

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, _, done, _, _ = env.step(action)

    print("   Backtest fertig.")

    # --- NEU: DATEN EXPORT FÜR DASHBOARD ---
    print("   Erstelle Dashboard-Daten (Extended)...")

    # Listen auf gleiche Länge bringen (Sicherheitsnetz)
    min_len = min(len(env.date_history), len(env.weights_history), len(env.portfolio_history))

    # 1. Metriken DataFrame
    metrics_df = pd.DataFrame({
        'Portfolio_Value': env.portfolio_history[:min_len],
        'Daily_Return': env.daily_return_history[:min_len],
        'Daily_Cost': env.cost_history[:min_len],
        'Turnover': env.turnover_history[:min_len],
        'Market_Avg_Return': env.mkt_return_history[:min_len]
    }, index=env.date_history[:min_len])

    # 2. Gewichte DataFrame
    # Spaltennamen generieren: Cash + Asset-Namen
    w_cols = ['Weight_Cash'] + [f"Weight_{name}" for name in env.asset_names]
    weights_df = pd.DataFrame(env.weights_history[:min_len], index=env.date_history[:min_len], columns=w_cols)

    # 3. Zusammenfügen und Speichern
    dashboard_df = pd.concat([metrics_df, weights_df], axis=1)

    csv_filename = "agent_dashboard_data_extended.csv"
    save_path = os.path.join(BASE_DIR, csv_filename)
    dashboard_df.to_csv(save_path)

    print(f"✅ Dashboard-Daten gespeichert unter: {save_path}")
    print(f"   Shape: {dashboard_df.shape}")
    # ---------------------------------------

    # --- QUANTSTATS REPORT ---
    if len(env.date_history) > len(env.portfolio_history):
        dates = env.date_history[:len(env.portfolio_history)]
    else:
        dates = env.date_history

    portfolio_series = pd.Series(env.portfolio_history, index=dates)
    returns = portfolio_series.pct_change().dropna()

    print("5. Lade Benchmark und erstelle Report...")
    try:
        qs.extend_pandas()
        benchmark = qs.utils.download_returns('^GSPC')
        benchmark = benchmark.loc[returns.index[0]:returns.index[-1]]
    except:
        print("⚠️ Konnte Benchmark nicht laden, erstelle Report ohne Benchmark.")
        benchmark = None

    report_file = "QuantStats_Report.html"
    qs.reports.html(returns, benchmark=benchmark, output=report_file, title="PPO Agent Evaluation")

    print(f"\n✅ Report erstellt: {report_file}")

    print("\n--- KEY METRICS ---")
    print(f"Cumulative Return: {qs.stats.comp(returns) * 100:.2f} %")
    print(f"CAGR:              {qs.stats.cagr(returns) * 100:.2f} %")
    print(f"Sharpe Ratio:      {qs.stats.sharpe(returns):.2f}")

    return report_file

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [8]:
run_evaluation()

1. Lade Daten...
2. Erstelle Environment...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


3. Lade Modell...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


4. Starte Backtest (das kann kurz dauern)...


...

✅ Dashboard-Daten gespeichert unter: /content/drive/MyDrive/02_DataScience_Quant/code/Seminar Deep Learning/PPO_Portfolio_Optimization/agent_dashboard_data_extended.csv
   Shape: (1007, 509)
5. Lade Benchmark und erstelle Report...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


✅ Report erstellt: QuantStats_Report.html

--- KEY METRICS ---
Cumulative Return: 268.31 %
CAGR:              38.62 %
Sharpe Ratio:      0.97


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


'QuantStats_Report.html'